# Budowa modelu do predykcji macierzy X na wartości wektora y
> Chcemy żeby model przewidywał rekord określający liczbę wystąpień słów na wartość 0/1 oznaczającą umeblowanie lub nie

### Zastosowany został model regresji logistycznej

In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# 1. Wczytanie danych

In [2]:
# wczytuje macierz i wektor ze sklasyfikowaną ręcznie zmienną określającą umeblowanie (1) lub nieumeblowanie+brak informacji (0)
X=pd.read_pickle('X.pkl')
y=pd.read_pickle('y.pkl')


In [4]:
X[:10,:10]

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [20]:
y[:5]

0    1
1    0
2    1
3    0
4    1
Name: furniture_class, dtype: int64

In [5]:
print(X.shape)
print(y.shape)

(8646, 100)
(8646,)


# 2. Podział na podpróbki
> wyjaśnienie: https://www.statystyczny.pl/grupa-uczaca-walidacyjna-i-testowa/

In [6]:
# dokonuje podziału zbioru na sety: training - uczący, validation - walidacyjny, test - testowy
X_train, X_test_all, y_train, y_test_all = train_test_split(X, y, test_size=0.50, random_state=0)
X_test, X_val, y_test, y_val = train_test_split(X_test_all, y_test_all, test_size=0.50, random_state=0)
# 50% to uczący, 25% walidacyjny, 25% testowy

![Alt text](https://developer.qualcomm.com/sites/default/files/attachments/learning_resources_03-03.png "a title")

In [7]:
# widać, że wszystkie próbki zachowują strukturę zbioru pierwotnego
print(y.groupby(y).count())
print(y_train.groupby(y_train).count())
print(y_val.groupby(y_val).count())
print(y_test.groupby(y_test).count())

furniture_class
0    2290
1    6356
Name: furniture_class, dtype: int64
furniture_class
0    1163
1    3160
Name: furniture_class, dtype: int64
furniture_class
0     553
1    1609
Name: furniture_class, dtype: int64
furniture_class
0     574
1    1587
Name: furniture_class, dtype: int64


# 3. Zastosowanie modelu regresji logistycznej
> https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [8]:
# utworzenie klasyfikatora (modelu) z zastosowaniem zadanych parametrów i pozostawieniem pozostałych domyślnych
clf_lr = LogisticRegression(solver='liblinear', C=1, random_state=0) 
# dopasowanie modelu na zbiorze uczącym
clf_lr.fit(X_train, y_train) # X - Training vector, y - target vector

LogisticRegression(C=1, random_state=0, solver='liblinear')

In [25]:
clf_lr.get_params() # wszystkie parametru modelu (łącznie z domyślnymi)

{'C': 1,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [10]:
# przewidujemy wartość dla całego zbioru X
# otrzymujemy predykcję zbioru X, sklasyfikowanego na wartości 0/1 mówiące nam czy dany rekord jest umeblowany czy nie
clf_lr.predict(X)[10:50]

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1], dtype=int64)

In [11]:
# oszacowanie prawdopodobieństwa dopasowania
probability = clf_lr.predict_proba(X) # pierwsza kolumna to nasze y=0, druga y=1
probability

array([[0.02634491, 0.97365509],
       [0.45673658, 0.54326342],
       [0.56143001, 0.43856999],
       ...,
       [0.00277152, 0.99722848],
       [0.00441459, 0.99558541],
       [0.52575878, 0.47424122]])

In [35]:
# możemy dokonać przeglądu prawdopodobieństwa, wyszukując rekordy, których prawdopodobieństwo poprawności dopasowania zawiera się w pewnym przedziale
np.where(np.logical_and(probability[:,0]>=0.45, probability[:,0]<=0.55))

(array([  81,  181,  292,  405,  411,  578,  599,  667,  747,  840,  846,
         886,  907,  914,  968, 1067, 1082, 1367, 1496, 1507, 1607, 1625,
        1691, 1709, 1752, 1936, 2152, 2319, 2321, 2433, 2503, 2531, 2542,
        2576, 2658, 2731, 2779, 2845, 2878, 3015, 3095, 3390, 3601, 3749,
        3761, 3799, 3851, 3906, 4012, 4026, 4045, 4229, 4365, 4382, 4571,
        4805, 4985, 5000, 5014, 5067, 5109, 5188, 5209, 5215, 5376, 5377,
        5455, 5477, 5527, 5533, 5604, 5688, 5961, 6004, 6149, 6180, 6222,
        6495, 6505, 6545, 6567, 6705, 6779, 6794, 6888, 6946, 7296, 7435,
        7498, 7499, 7603, 7672, 7695, 7831, 7849, 7887, 7910, 8114, 8218,
        8272, 8311, 8337, 8386, 8442, 8443, 8444, 8447, 8448], dtype=int64),)

In [36]:
# możemy podejrzeć konkretny rekord
probability[81]

array([0.45152285, 0.54847715])

In [42]:
# możemy sprawdzić ile jest rekordów z określonym prawdopodobieństwem
len(np.where(probability[:,0]>=0.85)[0])

1679

In [10]:
# wynik na zbiorze na którym się uczył jest bliski 1, ponieważ te dane zna w 100%
# score mówi nam o tym ile rekordów ma dobrze trafioną przewidywaną zmienną
# ile % ofert (X) model przewidział tak samo jak zostało to ręcznie sklasyfikowane (w wektorze y)
clf_lr.score(X_train, y_train) 


0.9949109414758269

### what is score?: 
> https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score

In [12]:
# wynik na zbiorze walidacyjnym, daje nam informację jak model radzi sobie z danymi, których teoretycznie nie widział
clf_lr.score(X_val, y_val)

0.7858464384828863

In [13]:
# wynik na zbiorze testowym to ostateczny sprawdzian jakości dopasowania modelu (na kolejnym nie znanym modelowi zbiorze)
clf_lr.score(X_test, y_test) # w 85% przypadków model przewidział poprawnie szacowaną wartość

0.7848218417399352

## jakie mogą być błędy na tym etapie?
> zbiór ze zbyt dużą liczbą zduplikowanych wartości pomiędzy zbiorem uczącym a testowym - przewiduje dobrze, bo dostaje dane które już zna

## jak poprawnie podzielić zbiór?
> by partitioning the available data into three sets, we drastically reduce the number of samples 
which can be used for learning the model, and the results can depend on a particular random choice 
for the pair of (train, validation) sets.

> A solution to this problem is a procedure called cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV

# 4. Cross validation
### metoda usprawniająca proces dopasowania modelu
> https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation </br>
> https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
### wracamy do momentu po kroku 1. (nie wykonujemy pkt. 2-3)

In [16]:
from sklearn.model_selection import cross_val_score
clf_lr = LogisticRegression(solver='liblinear', C=1, random_state=0)
# poniższa funkcja ma wbudowane dopasowanie (fit) modelu, więc nie ma fit wcześniej, ani nie wydzielamy zbioru testowego
scores_lr = cross_val_score(clf_lr, X, y, cv=5) 
scores_lr

array([0.75953757, 0.77803468, 0.78728324, 0.77572254, 0.76531792,
       0.72947977, 0.78703704, 0.74537037, 0.80092593, 0.76041667])

In [17]:
# możemy sprawdzić jaka jest ogólna wartość Accuracy dla modelu
print("%0.3f accuracy with a standard deviation of %0.2f" % (scores_lr.mean(), scores_lr.std())) 
# accuracy to średnia z wyników cross-validation

0.769 accuracy with a standard deviation of 0.02


![Alt text](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

### domyślnie cross_val_score liczy accuracy, ale możemy zastosować inne mierniki


In [88]:
scores_lr = cross_val_score(clf_lr, X, y, cv=5, scoring='f1_macro')
print(scores_lr)
print(scores_lr.mean())

[0.78807801 0.8195717  0.7800411  0.7672306  0.78327854]
0.7876399916083164


In [86]:
# wyświetlam wszystkie możliwe mierniki, które możemy zastosować w zależności od potrzeb
from sklearn.metrics import get_scorer_names
print(get_scorer_names())

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'positive_likelihood_ratio', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weight

### możemy też nie ograniczać oceny do jednego miernika
> https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

In [52]:
from sklearn.model_selection import cross_validate
scores_lr = cross_validate(clf_lr, X, y, cv=5, scoring=('accuracy', 'r2', 'neg_mean_squared_error'))
scores_lr


{'fit_time': array([0.77837205, 0.73545122, 0.68654013, 0.68720508, 0.81228209]),
 'score_time': array([0.03120041, 0.03182149, 0.01778722, 0.0312407 , 0.03125381]),
 'test_accuracy': array([0.82716763, 0.85887796, 0.83458647, 0.81781377, 0.83516484]),
 'test_r2': array([0.11209868, 0.27527409, 0.15052618, 0.06439072, 0.15349637]),
 'test_neg_mean_squared_error': array([-0.17283237, -0.14112204, -0.16541353, -0.18218623, -0.16483516])}

In [55]:
print('accuracy =', scores_lr['test_accuracy'].mean())


accuracy = 0.8347221321422721


# 5. GridSearchCV

### Powyższe podejścia były próbą trafienia ręcznie wyznaczonych parametrów modelu. Żeby proces ten przyspieszyć, możemy zastosować GridSearchCV do poszukiwania najlepszego zestawienia parametrów spośród narzuconych przestrzeni

> GridSearch ma wbudownay parametr do cros-walidacji

In [18]:
# zostawiamy fragment na zbiór testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [19]:
# ograniczamy przestrzeń poszukiwań do pewnego zestawu wartości poszczególnych parametrów
param_grid = [
              {'C': [0.1, 1], 
               'fit_intercept' : [False], 
               'class_weight' : [None],
               'random_state' : [0], 
               'solver' : ['lbfgs'] #, 'liblinear', 'newton-cg', 'sag', 'saga'
              } 
              ]

### GridSearchCV będzie przeszukiwał wszystkie możliwe kombinacje parametrów z nakreślonej przestrzeni i dopasowywał każdorazowo model

In [20]:
import time
import math
# wstawiam dodatkowo zliczenie czasu wykonywania dopasowania. im większa przestrzeń tym dłuższe poszukiwanie.
start = time.time()

clf = GridSearchCV(LogisticRegression(), param_grid, cv=5, n_jobs=5, verbose=2) 
clf.fit(X_train,y_train)

end = time.time()
elapsed = end-start
print(f'Fitting done in {math.trunc(elapsed/60)} min {math.trunc(round(elapsed%60,0))} sec')

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting done in 0 min 7 sec


In [21]:
# najlepszy zestaw parametrów
clf.best_params_

{'C': 1,
 'class_weight': None,
 'fit_intercept': False,
 'random_state': 0,
 'solver': 'lbfgs'}

In [22]:
# zestawienie wyników poszczególnych dopasowań dla zadanych zestawów parametrów
pd.DataFrame.from_dict(clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_fit_intercept,param_random_state,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.078232,0.039461,0.000000,0.000000,0.1,None,False,0,lbfgs,"{'C': 0.1, 'class_weight': None, 'fit_intercep...",0.784682,0.768619,0.775127,0.783803,0.785249,0.779496,0.006572,2
1,0.140610,0.038257,0.003127,0.006254,1,None,False,0,lbfgs,"{'C': 1, 'class_weight': None, 'fit_intercept'...",0.785405,0.773680,0.778742,0.788865,0.779465,0.781231,0.005330,1


In [17]:
clf.best_estimator_

LogisticRegression(C=0.1, random_state=0)

In [23]:
clf.best_score_ # Mean cross-validated score of the best_estimator

0.7812313225416807

In [24]:
# wynik na zbiorze testowym
clf.score(X_test, y_test)

0.7820809248554913

# 6. Co teraz?
### testujemy budowę modelu

1. zastosujmy jeden wspólny random state = 0
2. zastosujmy SVM i RandomForest, ew. Naive Bayes
2. testujcie z użyciem cross-validacji i bez
3. analizujcie wyniki na poziomie jednostkowym oraz mierniki modelu
4. zmieniajcie parametry
6. możecie eksperymentować z RandomizedSearchCV lub BayesSearchCV

* https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html
